In [124]:
DROP TABLE Conversion;
DROP TABLE CustomerConversion;
DROP TABLE ORDERJOIN;


DROP TABLE

DROP TABLE

DROP TABLE

Total execution time: 00:00:00.170

In [110]:
--- Creating a temporary table to join select relevant columns from the events table
CREATE TEMP TABLE Conversion AS 
    SELECT 
        internet_session_id,
        customer_id,
       article_ID,
       device_category_desc,
        MAX(CASE WHEN action_type_desc = 'purchase' THEN 1 ELSE 0 END) AS conversion,
        SUM(CASE WHEN action_type_desc = 'product detail view' THEN 1 ELSE 0 END) AS pdview
    FROM 
        events
    GROUP BY 
        internet_session_id,
        customer_id,
        device_category_desc,
       article_ID;

SELECT 805384

Total execution time: 00:00:02.688

In [111]:
--- Creating a temporary table to join relevant columns from the customers table onto the previous created Conversion table
CREATE TEMP TABLE CustomerConversion AS 
    SELECT
        C.article_id,
        C.internet_session_id, 
        C.customer_id, 
        C.conversion,
        C.pdview,
        C.device_category_desc, 
        cus.gender_code, 
        cus.geom_household_age,
        cus.geom_household_income,
        cus.geom_consumption_frequency,
        cus.geom_clothing_budget
    FROM 
        Conversion AS C
    LEFT JOIN 
        customers AS cus 
    ON 
        C.customer_id = cus.customer_id


SELECT 805384

Total execution time: 00:00:01.616

In [117]:
--- Creating a temporary table to join relevant columns from the article table onto the previous created CustomerConversion table: THIS IS OUR FINAL TABLE THAT WE EXPORT!
CREATE TEMP TABLE ORDERJOIN AS 
    SELECT
        cc.internet_session_id,
        cc.customer_id,
        RIGHT(internet_session_id, 10) AS session_date,
        cc.conversion,
        cc.pdview,
        cc.article_id,
        art.brand_name,
        art.class_2,
        art.class_3,
        art.live_year,
        cc.device_category_desc,
        cc.gender_code,
        cc.geom_household_age,
        cc.geom_household_income,
        cc.geom_consumption_frequency,
        cc.geom_clothing_budget
    FROM 
        CustomerConversion AS cc
    LEFT JOIN 
        article AS art
    ON 
        cc.article_id = art.article_id

       

SELECT 805384

Total execution time: 00:00:01.780

In [121]:
--- Performing disaster checks for the final table

-- Calculating the conversion rate for 2022 based on unique session_id
select ((select cast(count(distinct(internet_session_id))as FLOAT)
from ORDERJOIN
where conversion = 1 AND session_date LIKE '2022%') / cast(count(distinct(internet_session_id))as FLOAT)) * 100 as conversion_2022
from ORDERJOIN
where session_date LIKE '2022%';

-- Calculating the conversion rate for 2021 based on unique session_id
select ((select cast(count(distinct(internet_session_id))as FLOAT)
from ORDERJOIN
where conversion = 1 AND session_date LIKE '2021%') / cast(count(distinct(internet_session_id))as FLOAT)) * 100 as conversion_2021
from ORDERJOIN
where session_date LIKE '2021%';

-- Calculating the total conversion rate based on unique session_id
select ((select cast(count(distinct(internet_session_id))as FLOAT)
from ORDERJOIN
where conversion = 1) / cast(count(distinct(internet_session_id))as FLOAT)) * 100 as conversion_overall
from ORDERJOIN;

-- Calculating the conversion rate for 2021 based on views
select (sum(conversion)/sum(pdview))*100 as cr_perview_2021 from ORDERJOIN
where session_date LIKE '2021%';

-- Calculating the conversion rate for 2022 based on views
select (sum(conversion)/sum(pdview))*100 as cr_perview_2022 from ORDERJOIN
where session_date LIKE '2022%';

-- Calculating the total conversion rate based on views
select (sum(conversion)/sum(pdview))*100 as cr_perview_overall from ORDERJOIN;

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

...

conversion_2021
9.997800708261707


conversion_overall
9.716542621072536


cr_perview_2021
7.94410116714282550400


cr_perview_2022
7.24296718911930938000


cr_perview_overall
7.52561041558256042900


In [122]:
--- Providing an overview of the amount of product detail views by purchase status (conversion) based on ORDERJOIN. To show the distribution of views based on whether a purchase was made or not
SELECT 
    CASE WHEN conversion = 1 THEN 'Yes' ELSE 'No' END AS purchase,
    COUNT(CASE WHEN pdview = 0 THEN 1 END) AS view0,
    COUNT(CASE WHEN pdview = 1 THEN 1 END) AS view1,
    COUNT(CASE WHEN pdview = 2 THEN 1 END) AS view2,
    COUNT(CASE WHEN pdview = 3 THEN 1 END) AS view3,
    COUNT(CASE WHEN pdview = 4 THEN 1 END) AS view4,
    COUNT(CASE WHEN pdview > 4 THEN 1 END) AS viewmore,
    COUNT(*) AS Total

FROM 
    ORDERJOIN
GROUP BY 
    purchase;

(2 row(s) affected)

Total execution time: 00:00:00.419

purchase,view0,view1,view2,view3,view4,viewmore,total
No,48317,658042,34119,4688,926,287,746379
Yes,20688,38226,64,22,4,1,59005
